In [ ]:

//Look at how to convert the born String to Integer:

LOAD CSV  WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/people.csv' AS row RETURN row.name, toInteger(row.born)  LIMIT 5;


//Look at how to convert the released String to Integer:

LOAD CSV  WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/movies.csv'  AS row RETURN row.title, row.tagline, toInteger(row.released) LIMIT 5;


//Look at how to turn multiple values into an array for roles:

LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/actors.csv' AS row
WITH row WHERE row.person = 'Hugo Weaving' RETURN split(coalesce(row.roles,""),";");

//Load the people data:

LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/people.csv'  AS row
CREATE (p:Person)
SET p.name = row.name, p.born = toInteger(row.born);

//Load the movie data:

LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/movies.csv'  AS row
CREATE (m:Movie)
SET m.title = row.title, m.released = toInteger(row.released), m.tagline = row.tagline;

//Create the index on Movie title:

CREATE CONSTRAINT UniqueMovieTitleConstraint FOR (m:Movie) REQUIRE m.title IS UNIQUE;

//Create the index on Person name:

CREATE CONSTRAINT UniquePersonNameConstraint FOR (p:Person) REQUIRE p.name IS UNIQUE;

//Load the directors:

LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/directors.csv' AS row
MATCH (m:Movie {title: row.movie})
MERGE (p:Person {name: row.person})
WITH m,p
MERGE (p)-[:DIRECTED]->(m);


//Load the acted in relationships:

LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/actors.csv'  as row
MATCH (p:Person) where p.name = row.person
MATCH (m:Movie) where m.title = row.movie
WITH row, p, m
MERGE (p)-[r:ACTED_IN]->(m)
SET r.roles = split(coalesce(row.roles,""),";");

//Look at what was loaded (partial):

MATCH (p:Person)--(m:Movie)
RETURN p,m LIMIT 25;

 
